**Exemplo Para Leitura Via Socket e Gravação em Arquivo**

In [2]:
#cria a seção a ser utiliza para estabelecer a conexão 
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.functions import window
from pyspark.sql.functions import col


spark = SparkSession \
    .builder \
    .appName("StructuredWorkWordCountToFile") \
    .getOrCreate()

22/12/28 22:31:37 WARN Utils: Your hostname, Deboras-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.9 instead (on interface en0)
22/12/28 22:31:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/28 22:31:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
#cria o dataframe que será responsável por ler cada uma das linhas recebidas através do localhost e porta 9999
# define a fonte (source) de dados
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .option('includeTimestamp', 'true')\
    .load()

# Divide as linhas recebidas em cada palavra
words = lines.select(
        explode(split(lines.value, ' ')).alias('word'),
        col("timestamp")#lines.timestamp
    )

# cria o novo dataframe a ser responsável por contar a quantidade de palavras digitadas
windowedCounts = words \
    .withWatermark("timestamp", "10 minutes") \
    .groupBy(
        window(col("timestamp"), "10 minutes", "5 minutes"),
        words.word) \
    .count()

22/12/28 22:32:03 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [4]:
#confere o tipo de dataframe
windowedCounts.isStreaming

True

In [6]:
# Define a consulta (query) e como deve ser realizada a saída (sink) para o stream criado 
query = windowedCounts\
    .writeStream\
    .outputMode('update')\
    .format('console')\
    .start()



query.awaitTermination() #aguarda até que a "streaming query" termine 

22/12/28 22:34:06 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/hn/zqmlf1cd4s3_llkx270x0kw80000gn/T/temporary-91d06093-4dff-4856-bfdb-bd5db7760e51. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/12/28 22:34:06 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+------+----+-----+
|window|word|count|
+------+----+-----+
+------+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+-----+-----+
|              window| word|count|
+--------------------+-----+-----+
|{2022-12-28 22:25...|teste|    1|
|{2022-12-28 22:30...|teste|    1|
+--------------------+-----+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+------+----+-----+
|window|word|count|
+------+----+-----+
+------+----+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+--------------------+----+-----+
|              window|word|count|
+--------------------+----+-----+
|{2022-12-28 22:30...|novo|    1|
|{2022-12-28 22:25...|novo|    1|
+--------------------+----+-----+



...

-------------------------------------------
Batch: 11
-------------------------------------------
+--------------------+------+-----+
|              window|  word|count|
+--------------------+------+-----+
|{2022-12-28 22:30...|exit()|    1|
|{2022-12-28 22:35...|exit()|    1|
+--------------------+------+-----+



-------------------------------------------
Batch: 12
-------------------------------------------
+------+----+-----+
|window|word|count|
+------+----+-----+
+------+----+-----+

22/12/28 22:35:20 WARN TextSocketMicroBatchStream: Stream closed by localhost:9999


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

22/12/29 02:15:28 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 308430 ms exceeds timeout 120000 ms
22/12/29 02:15:29 WARN SparkContext: Killing executors is not supported by current scheduler.


Para rodar é necessário ter o "nc" instalado e utiliar o comando antes de iniciar a célula anterior

nc -lk 9999